In [1]:
import os 
import glob
import warnings
warnings.filterwarnings("ignore")

from pprint import pprint
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

# from langchain.document_loaders import PyPDFLoader
# from langchain.llms import OpenAI
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.vectorstores import DocArrayInMemorySearch
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings



In [2]:
from dotenv import load_dotenv
load_dotenv()
os.environ['NVIDIA_API_KEY'] = os.getenv('NVIDIA_API_KEY')
os.environ['ADOBE_CLIENT_ID'] = os.getenv('ADOBE_CLIENT_ID')
os.environ['ADOBE_CLIENT_SECRET'] = os.getenv('ADOBE_CLIENT_SECRET')

In [3]:
from llms import llms 
from llms import set_api_key
set_api_key('NVIDIA_API_KEY', os.environ['NVIDIA_API_KEY'])
llm = llms.nvai_mixtral_llm


In [4]:
from adobe_pdf_services import AdobePDFParser
from adobe_pdf_services import AdobePDFLoader

In [5]:
parser = AdobePDFParser(
    client_id=os.environ['ADOBE_CLIENT_ID'],
    client_secret=os.environ['ADOBE_CLIENT_SECRET'],
    mode='chunks', # mode in ["json", "chunks", "data"]
)

In [6]:
data_dir = os.path.join('data')
file_lst = sorted(glob.glob(os.path.join(data_dir, '*.pdf')))
for item in file_lst:
    print(item.split('/')[-1])

NIPS-2017-attention-is-all-you-need-Paper.pdf


In [7]:
for item in file_lst:
    file_name = item.split('/')[-1]
    print(file_name)
    # if os.path.exists(os.path.join(target_dir, file_name)):
    #     continue
    # shutil.copy(item, target_dir)
    loader = AdobePDFLoader(item, parser=parser)
    chunks = loader.load()

NIPS-2017-attention-is-all-you-need-Paper.pdf


In [8]:
for chunk in chunks:
    pprint(chunk.page_content)

Attention Is All You Need 
Ashish Vaswani∗ Google Brain avaswani@google.com 
∗ 
Noam Shazeer
Google Brain 
noam@google.com 
Niki Parmar∗ Google Research nikip@google.com 
Jakob Uszkoreit∗ 
Google Research usz@google.com 
∗ 
Llion Jones
Google Research 
llion@google.com 
∗ 
† 
Aidan N. Gomez
University of Toronto 
aidan@cs.toronto.edu 
∗ 
Łukasz Kaiser
Google Brain 
lukaszkaiser@google.com 
∗ 
‡ 
Illia Polosukhin
illia.polosukhin@gmail.com

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that 
include an encoder and a decoder. The best performing models also connect the encoder and decoder through an 
attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention 
mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show
these models to be superior in quality while being more parallelizable and requiring significantly less time to 
train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing
best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model 
establishes a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs, a 
small fraction of the training costs of the best models from the literature.

Recurrent neural networks, long short-term memory [(<>)12] and gated recurrent [(<>)7] neural networks in 
particular, have been firmly established as state of the art approaches in sequence modeling and transduction 
problems such as language modeling and machine translation [(<>)29, (<>)2, (<>)5]. Numerous efforts have since 
continued to push the boundaries of recurrent language models and encoder-decoder architectures [(<>)31, (<>)21, 
(<>)13]. 
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started the 
effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and has 
been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head 
attention and the parameter-free position representation and became the other person involved in nearly every 
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and 
tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and 
efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and 
implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating 
our research. 
†Work performed while at Google Brain. 
‡Work performed while at Google Research. 
31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA. 
Recurrent models typically factor computation along the symbol positions of the input and output sequences. 
Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of
the previous hidden state ht−1 and the input for position t. This inherently sequential nature precludes 
parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints 
limit batching across examples. Recent work has achieved significant improvements in computational efficiency 
through factorization tricks [(<>)18] and conditional computation [(<>)26], while also improving model performance 
in case of the latter. The fundamental constraint of sequential computation, however, remains. 
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in 
various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences 
[(<>)2, (<>)16]. In all but a few cases [(<>)22], however, such attention mechanisms are used in conjunction with a
recurrent network. 
In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on 
an attention mechanism to draw global dependencies between input and output. The Transformer allows for 
significantly more parallelization and can reach a new state of the art in translation quality after being trained 
for as little as twelve hours on eight P100 GPUs.

The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [(<>)20], ByteNet 
[(<>)15] and ConvS2S [(<>)8], all of which use convolutional neural networks as basic building block, computing 
hidden representations in parallel for all input and output positions. In these models, the number of operations 
required to relate signals from two arbitrary input or output positions grows in the distance between positions, 
linearly for ConvS2S and logarithmically for ByteNet. This makes it more difficult to learn dependencies between 
distant positions [(<>)11]. In the Transformer this is reduced to a constant number of operations, albeit at the 
cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with 
Multi-Head Attention as described in section (<>)3.2. 
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single
sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a 
variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning 
task-independent sentence representations [(<>)4, (<>)22, (<>)23, (<>)19]. 
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-aligned recurrence and 
have been shown to perform well on simple-language question answering and language modeling tasks [(<>)28]. 
To the best of our knowledge, however, the Transformer is the first transduction model relying entirely on 
self-attention to compute representations of its input and output without using sequence-aligned RNNs or 
convolution. In the following sections, we will describe the Transformer, motivate self-attention and discuss its 
advantages over models such as [(<>)14, (<>)15] and [(<>)8].

Most competitive neural sequence transduction models have an encoder-decoder structure [(<>)5, (<>)2, (<>)29]. 
Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence of continuous 
representations z = (z1, ..., zn). Given z, the decoder then generates an output sequence (y1, ..., ym) of symbols 
one element at a time. At each step the model is auto-regressive [(<>)9], consuming the previously generated 
symbols as additional input when generating the next. 
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected 
layers for both the encoder and decoder, shown in the left and right halves of Figure (<>)1, respectively.

...

We employ three types of regularization during training: 
Residual Dropout We apply dropout [(<>)27] to the output of each sub-layer, before it is added to the sub-layer 
input and normalized. In addition, we apply dropout to the sums of the embeddings and the positional encodings in 
both the encoder and decoder stacks. For the base model, we use a rate of Pdrop = 0.1. 
Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the English-to-German
and English-to-French newstest2014 tests at a fraction of the training cost. 
|                                 | Model     | BLEU     | Unnamed: 2   | Training Cost (FLOPs)     |
|:--------------------------------|:----------|:---------|:-------------|:--------------------------|
| nan                             | EN-DE     | EN-FR    | EN-DE        | EN-FR                     |
| ByteNet [15]                    | 23.75     | nan      | nan          | nan                       |
| Deep-Att + PosUnk [32]          | nan       | 39.2     | nan          | 1.0 · 1020                |
| GNMT + RL [31]                  | 24.6      | 39.92    | 2.3 · 1019   | 1.4 · 1020                |
| ConvS2S [8]                     | 25.16     | 40.46    | 9.6 · 1018   | 1.5 · 1020                |
| MoE [26]                        | 26.03     | 40.56    | 2.0 · 1019   | 1.2 · 1020                |
| Deep-Att + PosUnk Ensemble [32] | nan       | 40.4     | nan          | 8.0 · 1020                |
| GNMT + RL Ensemble [31]         | 26.30     | 41.16    | 1.8 · 1020   | 1.1 · 1021                |
| ConvS2S Ensemble [8]            | 26.36     | 41.29    | 7.7 · 1019   | 1.2 · 1021                |
| Transformer (base model)        | 27.3      | 38.1     | 3.3 · 1018   | nan                       |
| Transformer (big)               | 28.4      | 41.0     | 2.3 · 1019   | nan                       |
Label Smoothing During training, we employed label smoothing of value ls = 0.1 [(<>)30]. This hurts perplexity, as
the model learns to be more unsure, but improves accuracy and BLEU score.

On the WMT 2014 English-to-German translation task, the big transformer model (Transformer (big) in Table (<>)2) 
outperforms the best previously reported models (including ensembles) by more than 2.0 BLEU, establishing a new 
state-of-the-art BLEU score of 28.4. The configuration of this model is listed in the bottom line of Table (<>)3. 
Training took 3.5 days on 8 P100 GPUs. Even our base model surpasses all previously published models and ensembles,
at a fraction of the training cost of any of the competitive models. 
On the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of 41.0, outperforming all 
of the previously published single models, at less than 1/4 the training cost of the previous state-of-the-art 
model. The Transformer (big) model trained for English-to-French used dropout rate Pdrop = 0.1, instead of 0.3. 
For the base models, we used a single model obtained by averaging the last 5 checkpoints, which were written at 
10-minute intervals. For the big models, we averaged the last 20 checkpoints. We used beam search with a beam size 
of 4 and length penalty α = 0.6 [(<>)31]. These hyperparameters were chosen after experimentation on the 
development set. We set the maximum output length during inference to input length + 50, but terminate early when 
possible [(<>)31]. 
Table (<>)2 summarizes our results and compares our translation quality and training costs to other model 
architectures from the literature. We estimate the number of floating point operations used to train a model by 
multiplying the training time, the number of GPUs used, and an estimate of the sustained single-precision 
floating-point capacity of each GPU 5 (<>).

To evaluate the importance of different components of the Transformer, we varied our base model in different ways, 
measuring the change in performance on English-to-German translation on the development set, newstest2013. We used 
beam search as described in the previous section, but no checkpoint averaging. We present these results in Table 
(<>)3. 
In Table (<>)3 rows (A), we vary the number of attention heads and the attention key and value dimensions, keeping 
the amount of computation constant, as described in Section (<>)3.2.2. While single-head attention is 0.9 BLEU 
worse than the best setting, quality also drops off with too many heads. 
5We used values of 2.8, 3.7, 6.0 and 9.5 TFLOPS for K80, K40, M40 and P100, respectively. 
Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base model. All 
metrics are on the English-to-German translation development set, newstest2013. Listed perplexities are 
per-wordpiece, according to our byte-pair encoding, and should not be compared to per-word perplexities. 
|    | Unnamed: 0   | N                                         |   dmodel   |   dff   |   h   |   dk   |   dv   | 
P  drop   |   ls   | train steps     |   PPL  (dev)   |   BLEU  (dev)   |   params  ×106   |
|---:|:-------------|:------------------------------------------|-----------:|--------:|------:|-------:|-------:|-
-----------:|--------:|:----------------|---------------:|----------------:|-----------------:|
|  0 | base         | 6                                         |        512 |    2048 |     8 |     64 |     64 | 
0.1 |    0.1  | 100K            |           4.92 |            25.8 |            65    |
|  1 | nan          | nan                                       |        nan |     nan |     1 |    512 |    512 | 
nan   |  nan    | nan             |           5.29 |            24.9 |           nan    |
|  2 | (A)          | nan                                       |        nan |     nan |   nan |      4 |    128 | 
128   |  nan    | nan             |         nan    |             5   |            25.5  |
|  3 | nan          | nan                                       |         16 |      32 |    32 |    nan |    nan | 
nan   |    4.91 | 25.8            |         nan    |           nan   |           nan    |
|  4 | nan          | nan                                       |        nan |     nan |    32 |     16 |     16 | 
nan   |  nan    | nan             |           5.01 |            25.4 |           nan    |
|  5 | nan          | nan                                       |        nan |     nan |   nan |     16 |    nan | 
nan   |  nan    | nan             |           5.16 |            25.1 |            58    |
|  6 | (B)          | nan                                       |        nan |     nan |   nan |     32 |    nan | 
nan   |  nan    | nan             |           5.01 |            25.4 |            60    |
|  7 | nan          | 2                                         |        nan |     nan |   nan |    nan |    nan | 
nan   |  nan    | nan             |           6.11 |            23.7 |            36    |
|  8 | nan          | 4                                         |        nan |     nan |   nan |    nan |    nan | 
nan   |  nan    | nan             |           5.19 |            25.3 |            50    |
|  9 | nan          | 8                                         |        nan |     nan |   nan |    nan |    nan | 
nan   |  nan    | nan             |           4.88 |            25.5 |            80    |
| 10 | (C)          | nan                                       |        256 |     nan |   nan |     32 |     32 | 
nan   |  nan    | nan             |           5.75 |            24.5 |            28    |
| 11 | nan          | nan                                       |       1024 |     nan |   nan |    128 |    128 | 
nan   |  nan    | nan             |           4.66 |            26   |           168    |
| 12 | nan          | nan         

In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, 
replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed 
self-attention. 
For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or
convolutional layers. On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we 
achieve a new state of the art. In the former task our best model outperforms even all previously reported 
ensembles. 
We are excited about the future of attention-based models and plan to apply them to other tasks. We plan to extend 
the Transformer to problems involving input and output modalities other than text and to investigate local, 
restricted attention mechanisms to efficiently handle large inputs and outputs such as images, audio and video. 
Making generation less sequential is another research goals of ours. 
The code we used to train and evaluate our models is available at 
(<https://github.com/tensorflow/tensor2tensor>)https://github.com/ 
(<https://github.com/tensorflow/tensor2tensor>)tensorflow/tensor2tensor. 
Acknowledgements We are grateful to Nal Kalchbrenner and Stephan Gouws for their fruitful comments, corrections and
inspiration.

[1]  Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E Hinton. Layer normalization. arXiv preprint arXiv:1607.06450, 
2016. 
[2]  Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly learning to align 
and translate. CoRR, abs/1409.0473, 2014. 
[3]  Denny Britz, Anna Goldie, Minh-Thang Luong, and Quoc V. Le. Massive exploration of neural machine translation 
architectures. CoRR, abs/1703.03906, 2017. 
[4]  Jianpeng Cheng, Li Dong, and Mirella Lapata. Long short-term memory-networks for machine reading. arXiv 
preprint arXiv:1601.06733, 2016. 
[5]  Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk, and Yoshua Bengio. 
Learning phrase representations using rnn encoder-decoder for statistical machine translation. CoRR, abs/1406.1078,
2014. 
[6]  Francois Chollet. Xception: Deep learning with depthwise separable convolutions. arXiv preprint 
arXiv:1610.02357, 2016. 
[7]  Junyoung Chung, Çaglar Gülçehre, Kyunghyun Cho, and Yoshua Bengio. Empirical evaluation of gated recurrent 
neural networks on sequence modeling. CoRR, abs/1412.3555, 2014. 
[8]  Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin. Convolu-tional sequence to 
sequence learning. arXiv preprint arXiv:1705.03122v2, 2017. 
[9]  Alex Graves. Generating sequences with recurrent neural networks. arXiv preprint arXiv:1308.0850, 2013. 
[10]  Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. Deep residual learning for image recognition. In 
Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, pages 770–778, 2016. 
[11]  Sepp Hochreiter, Yoshua Bengio, Paolo Frasconi, and Jürgen Schmidhuber. Gradient flow in recurrent nets: the 
difficulty of learning long-term dependencies, 2001. 
[12]  Sepp Hochreiter and Jürgen Schmidhuber. Long short-term memory. Neural computation, 9(8):1735–1780, 1997. 
[13]  Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, and Yonghui Wu. Exploring the limits of 
language modeling. arXiv preprint arXiv:1602.02410, 2016. 
[14]  Łukasz Kaiser and Ilya Sutskever. Neural GPUs learn algorithms. In International Conference on Learning 
Representations (ICLR), 2016. 
[15]  Nal Kalchbrenner, Lasse Espeholt, Karen Simonyan, Aaron van den Oord, Alex Graves, and Ko-ray Kavukcuoglu. 
Neural machine translation in linear time. arXiv preprint arXiv:1610.10099v2, 2017. 
[16]  Yoon Kim, Carl Denton, Luong Hoang, and Alexander M. Rush. Structured attention networks. In International 
Conference on Learning Representations, 2017. 
[17]  Diederik Kingma and Jimmy Ba. Adam: A method for stochastic optimization. In ICLR, 2015. 
[18]  Oleksii Kuchaiev and Boris Ginsburg. Factorization tricks for LSTM networks. arXiv preprint arXiv:1703.10722,
2017. 
[19]  Zhouhan Lin, Minwei Feng, Cicero Nogueira dos Santos, Mo Yu, Bing Xiang, Bowen Zhou, and Yoshua Bengio. A 
structured self-attentive sentence embedding. arXiv preprint arXiv:1703.03130, 2017. 
[20]  Samy Bengio Łukasz Kaiser. Can active memory replace attention? In Advances in Neural Information Processing 
Systems, (NIPS), 2016. 
[21]  Minh-Thang Luong, Hieu Pham, and Christopher D Manning. Effective approaches to attention-based neural 
machine translation. arXiv preprint arXiv:1508.04025, 2015. 
[22]  Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention model. In 
Empirical Methods in Natural Language Processing, 2016. 
[23]  Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive summarization. 
arXiv preprint arXiv:1705.04304, 2017. 
[24]  Ofir Press and Lior Wolf. Using the output embedding to improve language models. arXiv preprint 
arXiv:1608.05859, 2016. 
[25]  Rico Sennrich, Barry Haddow, and Alexandra Birch. Neural machine translation of rare words with subword 
units. arXiv preprint arXiv:1508.07909, 2015. 
[26]  Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton, 

In [9]:
# emb = OpenAIEmbeddings()
emb = NVIDIAEmbeddings(model="nvolveqa_40k")
vdb = DocArrayInMemorySearch.from_documents(chunks, emb)

qas = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vdb.as_retriever(),
    return_source_documents=True,
)

In [13]:
# question = "What is the author list?"
# question = "What is the title of the paper?"
question = "What is the attention mechanism used in the model?"
response = qas({"query": question})
pprint(response["result"])


The Transformer model you've described uses a multi-head self-attention mechanism. In this mechanism, keys, values,
and queries all come from the same place in the model, allowing each position to attend to all positions in the 
same sequence. This is different from encoder-decoder attention, where queries come from the decoder and keys and 
values come from the encoder output, allowing every position in the decoder to attend over all positions in the 
input sequence. Multi-head attention is used in three different ways in the Transformer model: encoder-decoder 
attention, encoder self-attention, and decoder self-attention. The self-attention mechanism is used to compute 
representations of the input and output sequences without using sequence-aligned RNNs or convolution.